How to use gulp?

In [1]:
var execSync = require('child_process').execSync;
try {
    require.resolve('gulp');
    require.resolve('gulp-runtime');
    require.resolve('gulp-replace');
    require.resolve('gulp-tap');
} catch (e) {
    execSync('npm install "github:gulpjs/gulp#4.0" gulp-runtime gulp-replace gulp-tap');
}
var gulp = require('gulp');
var tap = require('gulp-tap');
var replace = require('gulp-replace');

// rename the project
gulp.task('build :src :dest', function (files, project) {
    return gulp
        .src(files, {cwd: project})
        .pipe(replace('appId: \'my-app-id\'', 'appId: \'jupytangular-module-server\''))
        .pipe(tap(function(file) {
            console.log(file.contents.toString());
        }))
});

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/universal';

gulp.task('default', () => gulp.task(['build :src :dest'])(['**/app.browser.module.ts'], project));

$$.async()
try {
gulp.series(() => gulp.task(['default'])(), function (done) {
    done();
    $$.sendResult();
})();
} catch (e) {
    $$.sendError(e);
}

// angular
import { NgModule } from '@angular/core';
import { BrowserModule } from '@angular/platform-browser';

// libs
import { BrowserStateTransferModule, DEFAULT_STATE_ID } from '@ngx-universal/state-transfer';
import { CACHE } from '@ngx-cache/core';
import { BrowserCacheModule, MemoryCacheService, STATE_ID } from '@ngx-cache/platform-browser';

// modules & components
import { AppModule } from './app.module';
import { AppComponent } from './app.component';

@NgModule({
  bootstrap: [AppComponent],
  imports: [
    BrowserModule.withServerTransition({
      appId: 'jupytangular-module-server'
    }),
    BrowserStateTransferModule.forRoot(),
    BrowserCacheModule.forRoot([
      {
        provide: CACHE,
        useClass: MemoryCacheService
      },
      {
        provide: STATE_ID,
        useValue: DEFAULT_STATE_ID
      }
    ]),
    AppModule
  ]
})
export class AppBrowserModule {
}



How to convert a gulp task to a Promise?

How to use a Promise as a gulp task?



In [2]:

// I wish Linus Torvolds had absolute power over everything javascript(+derivatives) and he could just do a massive purge of all the bullshit.
var tasksToPromise = (tasks) => {
    try {
        return new Promise((resolve, reject) => {
            gulp.series.apply(gulp, tasks.concat([function (done) {
                resolve(done());
            }]))();
        });
    } catch(e) {
        reject(e);
    }
};
tasksToPromise;



[Function: tasksToPromise]

In [3]:
var fs = require('fs');
var path = require('path');
var replace = require('gulp-replace');
var gulp = require('gulp');

// add automock to project for callback services
var client, renderer, server, core;

var applyUniversal = (project) => {
    // gulp.task('theme',
    //    gulp.src('./some/dir/**/*.js', {cwd: 'path/to/project'})
    //        .src('./specific-file.ts')) // already within cwd in chain?


    // TODO: Add material theme for the tri-color-wheel that updates the component immediately
    // add material theme?
    gulp.task('theme', function () {
        // TODO: always check for return statements here
        return gulp
            .src('**/client/assets/sass/layout.scss', {cwd: project})
            // does this prepend?
            .pipe(tap(function(file) {
                // TODO: detect if the color set is already loaded and create color classes?
                file.contents = Buffer.concat([new Buffer(`
@import '~@angular/material/_theming';

@include mat-core();

$candy-app-primary: mat-palette($mat-indigo);
$candy-app-accent: mat-palette($mat-pink, A200, A100, A400);

// The warn palette is optional (defaults to red).
$candy-app-warn: mat-palette($mat-red);

// Create the theme object (a Sass map containing all of the palettes).
$theme: mat-light-theme($candy-app-primary, $candy-app-accent, $candy-app-warn);

@include angular-material-theme($theme);

    `), file.contents]);
            }))
            .pipe(gulp.dest(project));
    });
    
    // TODO: convert all interpreted searches to gulp tasks by scanning notebooks for valid outputs
    gulp.task('importer', function() {
        return require('../Core')
            .interpretAll([
                'mock audit',
                'module promisifty socket.io',
                'promisifiy socket.io server',
                'sockify server running',
                'render Angular modules',
                'core modules'
            ])
            .then(r => {
                client = r[0].code + r[1].code;
                server = r[2].code + r[3].cone;
                renderer = r[4].code;
                core = r[5].code;
            });
    });

    // client mock generator library to create socketio callbacks
    gulp.task('sockify client', function(cb){
        fs.writeFile(path.join(project, 'src/client/app/sockify-client.js'), client, cb);
    });
    
    // add shared module to app.module
    gulp.task('core', function(cb){
        fs.writeFile(path.join(project, 'src/client/app/core.module.ts'), core, cb);
    });

    // add server mock reporter to project
    gulp.task('sockify server', function(cb){
        fs.writeFile(path.join(project, 'src/server/sockify-server.js'), server, cb);
    });

     // add server renderer to project
    gulp.task('render service', function(cb){
        fs.writeFile(path.join(project, 'src/server/render-service.ts'), renderer, cb);
    });
    gulp.task('renderer', function() {
        return gulp.src(['**/server/render-service.js'], {cwd: project})
            // add mock socket server to server/render-server or app.server.module
   // TODO: pipe server.ts to add log provider for log service    
            .pipe(replace(
             'provide: APP_BASE_HREF,',
              `
                                    provide: AuthService,
                                    useValue: require('./sockify-server').sockifyRequire(AuthService, 'AuthService')
                                },
                                {
                                    provide: PlatformLocation,
                                    useValue: require('./sockify-server').sockifyRequire(PlatformLocation, 'PlatformLocation')
                                },
                                {
                                    provide: LogService,
                                    useValue: require('./sockify-server').sockifyRequire(LogService, 'LogService')
                                },
                                {
    `))
            .pipe(gulp.dest(project))
    });
    
    // export notebook code to files/pipes
    gulp.task('export notes',
        gulp.series('importer', 'sockify client', 'core', 'sockify server', 'render service', 'renderer'));
   
    // tell webpack to build the renderer seperately
    gulp.task('webpack config', function() {
        return gulp.src(['**/config/webpack.config.js'], {cwd: project})
            .pipe(replace('? webpackConfig.universal.server.prod(root, settings)', `
                ? webpackMerge(webpackConfig.universal.server.prod(root, settings), {
                    entry: {
                        'render-service': root(\`$\{settings.paths.src.server.root}/render-service.ts\`),
                        'server': root(\`$\{settings.paths.src.server.root}/server.ts\`)
                    },
                    output: {
                        filename: '[name].js'
                    }
                })
`))
            .pipe(gulp.dest(project));
    });
    
    // replace app module module with these?
    gulp.task('app module', function() {
        return gulp.src(['**/client/app/app.module.ts'], {cwd: project})
            // add some dependencies
            .pipe(replace('import { AppComponent } from \'./app.component\'', `;
import { AppComponent } from './app.component';
import { SharedModule, sharedModules } from './core.module';
import { Observable } from 'rxjs/Observable';
`))
            // replace the configuration loader with a Mock
            .pipe(replace(
              'const browserLoader = new ConfigHttpLoader(http, \'./assets/config.json\')',
              `
    const browserLoader = {
        loadSettings: () => Promise.resolve(
            (typeof window !== 'undefined' ? window as any : {}).CONFIG
            || {})
    } as ConfigHttpLoader;
`))
            // replace router
            .pipe(replace(
              'RouterModule.forRoot(routes)',
              `
        RouterModule.forRoot(routes, {initialNavigation: 'enabled', useHash: false})
`))
            // don't care about meta for this
            .pipe(replace(/MetaLoader\s*\{[\s\S]*?\}\);/ig,
              `MetaLoader {
        return new MetaStaticLoader();
`))
            .pipe(replace(/TranslateLoader\s\{[\s\S]*?\}/ig, 
                          `TranslateLoader {
    return {
        getTranslation: lang => Observable.of(
            (typeof window !== 'undefined' ? <any>window : {}).TRANSLATIONS)
    } as TranslateLoader;
}`))
            .pipe(gulp.dest(project));
    });

    // add socket-client provider for auth module to app.browser.module
    gulp.task('browser module', function() {
      return gulp.src(['**/client/app/app.browser.module.ts'], {cwd: project})
        .pipe(replace('import { AppComponent } from \'./app.component\'', `
import { AppComponent } from './app.component';
import { BrowserAnimationsModule } from '@angular/platform-browser/animations';
`))
        .pipe(replace(
          /\)\],\s*AppModule/ig,
          `)],
        AppModule,
        BrowserAnimationsModule,
    ],
    providers: [
        {
            provide: AuthService,
            useValue: require('./sockify-client').sockifyClient(AuthService, 'AuthService'),
        },
        {
            provide: PlatformLocation,
            useValue: require('./sockify-client').sockifyClient(PlatformLocation, 'PlatformLocation'),
        },
`))
        .pipe(gulp.dest(project))
    });
    
    // replace app component
    gulp.task('app component', function() {
      return gulp.src(['**/client/app/app.component.html'], {cwd: project})
        .pipe(replace(
         /[\s\S]*<router-outlet><\/router-outlet>.*/ig, '<router-outlet></router-outlet>'))
        .pipe(gulp.dest(project))
    });
    
    // replace routes with lazy modules from other projects
    gulp.task('app router', function() {
      return gulp.src(['**/client/app/app.routes.ts'], {cwd: project})
        .pipe(replace(
         /Routes = \[[\s\S].*?\];/ig, `Routes = [
    {
        path: '',
        children: [
            {
                path: 'auth',
                loadChildren: './auth/auth.module#AuthModule'
            }
        ],
        canActivateChild: [ MetaGuard ],
        data: {
            i18n: {
                isRoot: true
            }
        }
    },
    {
        path: 'change-language/:languageCode',
        component: ChangeLanguageComponent
    },
    {
        path: '**',
        redirectTo: '',
        pathMatch: 'full'
    }
`))
        .pipe(gulp.dest(project))
    });
    
    // replace package.json
    // TODO: automatically handle package resolution errors:
    // in *.js Can't resolve 'socket.io-client/package'
    // in *.ts Cannot find name 'JwtHelper' -> trace back to import statements?
    gulp.task('app package', function() {
      return gulp.src(['**/package.json'], {cwd: project})
        .pipe(replace(
         /"@angular\/common":.*?,/ig, `
    "@angular/common": "~4.2.0",
    "socket.io-client": "latest",
    "@types/socket.io-client": "latest",
    "automock": "latest",
    "angular2-jwt/angular2-jwt": "latest",
`))
        .pipe(gulp.dest(project))
    });
    
    // TODO: try new versions of packages to act like greenkeeper.io
        
    // gulp apply everything
    gulp.task('apply universal',
        gulp.series('theme',
                    'export notes',
                    'webpack config',
                    'app module',
                    'browser module',
                    'app component',
                    'app router',
                    'app package'));
    
    return tasksToPromise(['apply universal'], []);
};
applyUniversal;



'var execSync = require(\'child_process\').execSync;\ntry {\n    require.resolve(\'automock\');\n    require.resolve(\'bluebird\');\n} catch (e) {\n    execSync(\'npm install automock bluebird\');\n}\n\nvar Promise = require(\'bluebird\');\nvar automock = require(\'automock\');\nvar promisifyMock = (req, dep) => {\n    let ctx;\n    ctx = automock.mockValue(req, {\n        stubCreator: (name) => {\n            var orig = Promise.promisify(req[name.split(\'.\')[1]], {\n                multiArgs: true,\n                context: req\n            });\n            //console.log(\'create stub \' + name);\n            return function () {\n                console.log(name + \' (\' + arguments[0] + \') in \' + JSON.stringify(dep));\n                return orig.apply(null, arguments);\n            };\n        }\n    });\n    return ctx;\n};\npromisifyMock;var execSync = require(\'child_process\').execSync;\ntry {\n    require.resolve(\'automock\');\n    require.resolve(\'socket.io\');\n} catch 

Apply universal to angular project?

In [ ]:

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/universal';
if(!fs.existsSync(project)) {
    fs.mkdirSync(project);
}

$$.async()
applyUniversal(project)
    .then(r => $$.sendResult(client))
    .catch(e => $$.sendError(e));

